# LangChain


## 1. install

In [17]:
# source venv/bin/activate
# !pip install --upgrade pip
# !pip install fastapi
# !pip install -qU langchain-openai
# !pip install langchain-teddynote
# !pip install confluent-kafka
# !pip install -U langchain-community

  Using cached python_dotenv-1.0.1-py3-none-any.whl.metadata (23 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 10.8 MB/s eta 0:00:00a 0:00:01
Using cached python_dotenv-1.0.1-py3-none-any.whl (19 kB)


## 2. Test 1 : LangChain 
### LLM 개발, 모니터링 및 테스트 플랫폼
- 예상치 못한 최종 결과
- 에이전트가 루핑되는 이유
- 체인이 예상보다 느린 이유
- 에이전트가 각 단계에서 사용한 토큰 수
- reference : https://python.langchain.com/v0.2/docs/tutorials/chatbot/, https://wikidocs.net/250954
- LangSmith : https://smith.langchain.com/

### OpenAI Models
- temperature : 0-2, 낮을 수록 결정론적
- max_tokens : gpt-4o-mini, 16,384 / gpt-4o, 4,096 


In [15]:
import os
from dotenv import load_dotenv
from langchain_teddynote import logging
from langchain_openai import ChatOpenAI


# .env 파일에서 환경 변수 로드
load_dotenv()
langsmith_project = os.getenv("LANGCHAIN_PROJECT")

logging.langsmith(langsmith_project)

llm = ChatOpenAI(
    temperature=0.1,            # 창의성 (0.0 ~ 2.0)
    model_name="gpt-4o-mini",   # 모델명
)

# 질의내용
question = "외국인 노동자가 한국에서 일 할 수 있으려면 어떻게 해야해?"

# 질의
print(f"[답변]: {llm.invoke(question)}")

LangSmith 추적을 시작합니다.
[프로젝트명]
AI_labor
[답변]: content='외국인 노동자가 한국에서 일하기 위해서는 다음과 같은 절차를 따라야 합니다:\n\n1. **비자 종류 확인**: 한국에서 일하기 위해서는 적절한 비자를 취득해야 합니다. 일반적으로 외국인 노동자는 E-9(비숙련 노동자), E-7(전문직), E-5(전문직) 등의 비자를 신청할 수 있습니다. 자신의 직업과 상황에 맞는 비자 종류를 확인해야 합니다.\n\n2. **고용주 찾기**: 한국에서 일할 고용주를 찾아야 합니다. 고용주가 외국인 노동자를 고용할 수 있는 자격이 있는지 확인해야 하며, 고용계약서를 작성해야 합니다.\n\n3. **비자 신청**: 고용주와 계약을 체결한 후, 해당 비자를 신청해야 합니다. 비자 신청은 한국 대사관이나 영사관, 또는 한국의 출입국관리사무소에서 진행할 수 있습니다. 필요한 서류는 비자 종류에 따라 다르지만, 일반적으로 고용계약서, 여권, 사진, 건강검진 결과 등이 필요합니다.\n\n4. **비자 발급**: 비자 신청이 승인되면 비자를 발급받게 됩니다. 이때 비자 발급에 소요되는 시간은 비자 종류와 상황에 따라 다를 수 있습니다.\n\n5. **입국 및 체류 등록**: 한국에 입국한 후, 90일 이내에 출입국관리사무소에 가서 체류 자격 변경 및 등록을 해야 합니다.\n\n6. **근무 시작**: 모든 절차가 완료되면 한국에서 합법적으로 근무를 시작할 수 있습니다.\n\n각 단계에서 필요한 서류와 절차는 상황에 따라 다를 수 있으므로, 한국의 출입국관리사무소나 관련 기관에 문의하여 정확한 정보를 확인하는 것이 중요합니다.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 411, 'prompt_tokens': 24, 'total_tokens': 435, 'completion_tokens_details': {'reasoning_to

## 3. Test 2 : LangChain + Faiss
- OpenAIEmbeddings : OpenAI API를 통해 텍스트 -> 벡터화
- Docstore : LangChain에서 문서(document) 저장소 역할. FAISS 벡터 저장소를 사용할 때 실제 텍스트와 메타데이터를 저장
- InMemoryDocstore : 메모리 내에서 문서를 저장, 관리하는 Docstore의 구현체. 모든 문서를 메모리에 저장해 빠른 조회가 가능.
- Document : 텍스트 데이터(Body)와 그에 대한 메타데이터(출처 - Law name, Chapter, Title)를 담고 있는 객체
- VectorStore : 벡터화된 데이터 저장소(vector DB) 임베딩 저장, 유사도 검색, 임베딩 관리, ex) FAISS
- Retriever : vector_store를 통해 만듦. 사용자의 질문과 관련된 문서를 검색, 질문을 임베딩으로 변환해 유사한 문서 검색
- ChatPromptTemplate : LLM에 전달할 입력을 정의 ( 검색한 문서 + 사용자 질문 )
- RetrieverQA : 검색과 답변 생성을 통합하는 체인. Retriever를 사용해 검색된 문서들을 기반으로 LLM이 답변을 생성하도록 연결

### 흐름
1) FAISS로 구성된 vector store를 LangChain의 저장소 형태인 Docstore 형태로 저장하는데, 이때 InMemoryDocstore를 사용한다.
2) 사용자의 질문이 들어오면, 그 질문을 OpenAIEmbeddings를 이용해서 벡터화 한다.
3) 벡터화한 질문을 retriever를 통해 유사한 문서를 검색한다.
4) RetrieverQA(질의응답 체인)를 통해 검색된 문서와 사용자의 질문을 LLM 모델로 전달한다.

In [40]:
import faiss
import json
from dotenv import load_dotenv
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.vectorstores.faiss import FAISS
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_core.documents import Document

# .env 파일에서 환경 변수 로드
load_dotenv()
project_api_key = os.getenv("OPENAI_API_KEY")
organization_id = os.getenv("ORGANIZATION_ID")
project_id = os.getenv("PROJECT_ID")
langsmith_project = os.getenv("LANGCHAIN_PROJECT")

# OpenAI LLM 설정 (GPT-4o-mini)
llm = ChatOpenAI(
    temperature=0.1,            # 창의성 (0.0 ~ 2.0)
    model_name="gpt-4o-mini",   # 모델명
)

# 벡터 스토어 설정
def setup_faiss_vectorstore():
    # FAISS 인덱스 로드
    index = faiss.read_index("law_embeddings.index")

    # JSON 파일에서 문서 로드
    with open("law_embeddings.json", "r", encoding="utf-8") as f:
        docs_data = json.load(f)

    # OpenAI 임베딩 모델 설정
    embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

    # JSON 구조에 따라 Document 객체로 변환
    docs = []
    for item in docs_data:
        page_content = item.get('Body', '')  # 'Body' 필드 사용
        metadata = {
            "law": item.get('Law', ''),      # 'Law' 필드 사용
            "chapter": item.get('Chapter', ''),  # 'Chapter' 필드 사용
            "title": item.get('Title', '')   # 'Title' 필드 사용
        }
        doc = Document(page_content=page_content, metadata=metadata)
        docs.append(doc)

    # InMemoryDocstore에 문서 저장
    docstore = InMemoryDocstore({str(i): doc for i, doc in enumerate(docs)})

    # FAISS 벡터스토어 초기화
    vectorstore = FAISS(
        embedding_function=embeddings,      # 임베딩 생성기
        index=index,                        # 검색 인덱스
        docstore=docstore,                  # 문서 저장소    
        index_to_docstore_id={i: str(i) for i in range(len(docs))}  # 인덱스와 문서 ID 매핑
    )
    return vectorstore

# 질의응답 체인 설정
def setup_rag_chain(vectorstore):
    retriever = vectorstore.as_retriever()

    # 시스템 프롬프트 정의
    system_prompt = (
        "You are an AI chatbot specialized in Korean labor law. "
        "Use the following pieces of retrieved context to answer the question. "
        "If you don't know the answer, say that you don't know. "
        "Please explain in as much detail as possible."
        "Please mention exactly which law and title you referenced in the retrieved context."
        "\n\n"
        "{context}"
    )

    # 프롬프트 템플릿 생성
    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", system_prompt),
            ("human", "{input}"),
        ]
    )
    # 문서 조각을 결합하는 체인 생성
    question_answer_chain = create_stuff_documents_chain(llm, prompt)

    # 검색과 QA를 결합하는 체인 생성
    chain = create_retrieval_chain(retriever, question_answer_chain)

    return chain

# 질문 처리 및 답변 생성
def ask_labor_law_question(rag_chain, question):
    result = rag_chain.invoke({"input": question})  # 'input'을 사용하여 invoke 호출
    answer = result.get('answer', '답변을 찾을 수 없습니다.')  # 'answer' 키로 답변 가져오기
    source_docs = result.get('context', [])  # 참조된 문서 가져오기
    return answer, source_docs

# 메인 실행 함수
if __name__ == "__main__":
    # 벡터스토어 설정 (한 번만 설정)
    vectorstore = setup_faiss_vectorstore()

    # 질의응답 체인 설정 (한 번만 설정)
    rag_chain = setup_rag_chain(vectorstore)


    question = input("질문을 입력하세요 (종료하려면 'exit' 입력): ")

    answer, source_docs = ask_labor_law_question(rag_chain, question)

    # 출력 형식에 맞게 답변과 참조 문서 출력
    print(f"[답변]: {answer}\n")
    print("[참고 문서]:")
    if source_docs:
        for idx, doc in enumerate(source_docs, 1):
            print(f"{idx}. {doc.metadata['law']}, {doc.metadata['chapter']}, {doc.metadata['title']}")
    else:
        print("참조된 문서가 없습니다.")


question = "외국인 근로자가 한국에서 일 할 수 있으려면 어떻게 해야해?"

[답변]: 외국인 근로자가 한국에서 일하기 위해서는 몇 가지 절차와 요건을 충족해야 합니다. 관련 법률은 "외국인근로자법"입니다. 

1. **취업 비자 발급**: 외국인 근로자는 한국에서 합법적으로 일하기 위해 취업 비자를 받아야 합니다. 이 비자는 일반적으로 특정 직종이나 산업에 따라 다르게 발급됩니다. 외국인 근로자는 한국의 고용주로부터 일자리를 제안받고, 그에 따라 비자를 신청해야 합니다.

2. **고용주와의 계약**: 외국인 근로자는 한국의 고용주와 근로 계약을 체결해야 합니다. 이 계약은 근로 조건, 임금, 근무 시간 등을 명시해야 하며, 고용주는 외국인 근로자가 한국에서 일할 수 있도록 필요한 서류를 준비해야 합니다.

3. **고용노동부 신고**: 외국인 근로자를 모집하는 경우, 고용주는 고용노동부 장관에게 신고해야 합니다. 이는 외국인 근로자의 도입과 관련된 법적 요건 중 하나입니다. (출처: 외국인근로자법 제1항)

4. **체류 자격**: 외국인 근로자는 「출입국관리법」에 따라 취업활동을 할 수 있는 체류 자격을 받아야 합니다. 이 자격은 취업 분야나 체류 기간에 따라 다르게 정해질 수 있습니다.

5. **법적 요건 준수**: 외국인 근로자는 한국의 노동법 및 관련 법규를 준수해야 하며, 고용주는 외국인 근로자의 권리가 침해되지 않도록 노력해야 합니다. (출처: 외국인근로자법 제1항)

이러한 절차를 통해 외국인 근로자는 한국에서 합법적으로 일할 수 있습니다.

[참고 문서]:
1. 고용정책 기본법, 제5장 근로자의 고용촉진 및 사업주의 인력확보 지원, 제31조(외국인근로자의 도입)
2. 외국인근로자의 고용 등에 관한 법률, 제1장 총칙, 제2조(외국인근로자의 정의)
3. 교원의 노동조합 설립 및 운영 등에 관한 법률, 제3장 직업안정기관의 장 외의 자가 하는 직업소개사업, 직업정보제공사업, 근로자 모집 또는 근로자공급사업 등, 제30조(국외 취업자의 모집)
4. 직업안정법, 제3장 직업안정기관의 장 외의 자가 하는 직업소개사업, 직업정

In [ ]:
"""
[답변]: 외국인 노동자가 한국에서 일하기 위해서는 다음과 같은 절차를 따라야 합니다:

1. **고용허가 신청**: 사용자는 외국인 근로자를 고용하기 위해 고용노동부에 고용허가를 신청해야 합니다.

2. **근로계약 체결**: 고용허가를 받은 후, 사용자는 고용노동부령으로 정하는 표준근로계약서를 사용하여 외국인 근로자와 근로계약을 체결해야 합니다.

3. **신고 의무**: 국외에 취업할 근로자를 모집한 경우, 고용노동부장관에게 신고해야 합니다.

4. **비자 발급**: 외국인 근로자는 한국에서 일하기 위해 필요한 비자를 발급받아야 합니다.

이 외에도 구체적인 절차나 요구 사항은 관련 법률 및 대통령령에 따라 달라질 수 있습니다.

[참조 문서]:
- 고용정책 기본법 (제목: 제31조(외국인근로자의 도입))
- 교원의 노동조합 설립 및 운영 등에 관한 법률 (제목: 제30조(국외 취업자의 모집))
- 직업안정법 (제목: 제30조(국외 취업자의 모집))
- 외국인근로자의 고용 등에 관한 법률 (제목: 제9조(근로계약))
"""

## 4. Conversational RAG
- https://python.langchain.com/v0.2/docs/tutorials/qa_chat_history/